# ⚖️ Introduction to Fairness in ML

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Information</b><br>
<b>Level:</b> Intermediate<br>
<b>Duration:</b> 20 minutes<br>
<b>Dataset:</b> Adult Income (UCI) - simulated<br>
<b>Prerequisite:</b> 01_introducao/
</div>

## 🎯 Objectives
- ✅ Understand why fairness matters
- ✅ Learn about protected attributes
- ✅ Learn about the 15 fairness metrics
- ✅ Auto-detect sensitive attributes
- ✅ Execute first fairness test

## 📚 Why Does Fairness Matter?

### ❌ Real Cases of Bias in ML

#### 1. **Amazon - Recruiting System (2018)**
- **Problem**: Discriminated against female candidates
- **Cause**: Trained on biased historical data (mostly men)
- **Result**: System discontinued
- **Impact**: Millions in losses + reputational damage

#### 2. **COMPAS - Criminal Justice (2016)**
- **Problem**: Racial bias in recidivism prediction
- **Discovery**: ProPublica revealed disparities
- **Impact**: Black individuals unfairly received higher scores
- **Consequence**: Lawsuits and system review

#### 3. **Apple Card - Credit Limit (2019)**
- **Problem**: Women received lower limits than men
- **Evidence**: Public cases (Steve Wozniak)
- **Investigation**: Financial regulators
- **Lesson**: Even large companies can have bias

#### 4. **Facial Recognition**
- **Problem**: Lower accuracy for Black individuals
- **Cause**: Non-representative datasets
- **Risk**: Misidentification in security

### ✅ Why Validate Fairness?
1. **Ethical** - Don't discriminate against people
2. **Legal** - Compliance with regulations (EEOC, GDPR, LGPD)
3. **Business** - Avoid lawsuits and reputational damage
4. **Technical** - Fair models are better models

## 1️⃣ Setup and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from deepbridge import DBDataset, Experiment

# Configure visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")

# Create synthetic dataset (simulating Adult Income)
np.random.seed(42)
n = 2000

df = pd.DataFrame({
    'age': np.random.randint(18, 70, n),
    'education_years': np.random.randint(8, 20, n),
    'hours_per_week': np.random.randint(20, 60, n),
    'gender': np.random.choice(['Male', 'Female'], n, p=[0.6, 0.4]),
    'race': np.random.choice(['White', 'Black', 'Asian', 'Other'], n, 
                             p=[0.7, 0.15, 0.10, 0.05])
})

# Target: income >50k (with intentional bias for demonstration)
# Add bias: gender and race affect probability
base_prob = 0.3
prob = base_prob + \
       (df['age'] - 40) * 0.005 + \
       (df['education_years'] - 12) * 0.03 + \
       (df['hours_per_week'] - 40) * 0.01 + \
       (df['gender'] == 'Male') * 0.10 + \
       (df['race'] == 'White') * 0.05

df['high_income'] = (prob + np.random.normal(0, 0.1, n) > 0.5).astype(int)

print(f"📊 Dataset: {df.shape}")
print(f"\n📈 Target distribution:")
print(df['high_income'].value_counts(normalize=True))
print(f"\n👥 Distribution by gender:")
print(df['gender'].value_counts())
print(f"\n🌍 Distribution by race:")
print(df['race'].value_counts())

2025-11-04 22:47:20,847 - deepbridge.reports - INFO - Successfully imported radar chart fix
2025-11-04 22:47:20,851 - deepbridge.reports - INFO - Successfully patched EnhancedUncertaintyCharts.generate_model_metrics_comparison
2025-11-04 22:47:20,852 - deepbridge.reports - INFO - Successfully applied enhanced_charts patch
2025-11-04 22:47:20,856 - deepbridge.reports - INFO - Successfully loaded UncertaintyChartGenerator
2025-11-04 22:47:20,858 - deepbridge.reports - INFO - Successfully imported and initialized SeabornChartGenerator
2025-11-04 22:47:20,859 - deepbridge.reports - INFO - SeabornChartGenerator has_visualization_libs: True
2025-11-04 22:47:20,860 - deepbridge.reports - INFO - Available chart methods: ['bar_chart', 'boxplot_chart', 'coverage_analysis_chart', 'detailed_boxplot_chart', 'distribution_grid_chart', 'feature_comparison_chart', 'feature_importance_chart', 'feature_psi_chart', 'generate_encoded_chart', 'heatmap_chart', 'individual_feature_impact_chart', 'method_comp

## 2️⃣ Protected Attributes

### What are they?
Characteristics that **CANNOT be used** for discrimination:
- Gender
- Race/Ethnicity
- Age
- Religion
- Sexual Orientation
- Disability Status

### Regulations
- **USA**: EEOC (Equal Employment Opportunity Commission)
- **Europe**: GDPR (General Data Protection Regulation)
- **Brazil**: LGPD (General Data Protection Law)

In [2]:
# Visualize income distribution by protected attributes
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# By gender
gender_income = df.groupby('gender')['high_income'].mean()
axes[0].bar(gender_income.index, gender_income.values, 
            color=['skyblue', 'pink'], edgecolor='navy', alpha=0.7)
axes[0].set_title('High Income Rate by Gender', fontsize=13, fontweight='bold')
axes[0].set_ylabel('High Income Proportion', fontsize=11)
axes[0].set_ylim(0, 1)
axes[0].grid(axis='y', alpha=0.3)

# By race
race_income = df.groupby('race')['high_income'].mean()
axes[1].bar(race_income.index, race_income.values, 
            color='coral', edgecolor='darkred', alpha=0.7)
axes[1].set_title('High Income Rate by Race', fontsize=13, fontweight='bold')
axes[1].set_ylabel('High Income Proportion', fontsize=11)
axes[1].set_ylim(0, 1)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️  WARNING: We already see disparities in the data!")
print(f"   Male rate: {gender_income['Male']:.1%}")
print(f"   Female rate: {gender_income['Female']:.1%}")
print(f"   Difference: {abs(gender_income['Male'] - gender_income['Female']):.1%}")


⚠️  WARNING: We already see disparities in the data!
   Male rate: 48.2%
   Female rate: 29.6%
   Difference: 18.5%


/tmp/ipykernel_1828165/3448311929.py:24: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 3️⃣ Train Model and Create Experiment

In [3]:
# Prepare data (encode categorical variables)
df_encoded = df.copy()
df_encoded['gender_encoded'] = (df['gender'] == 'Male').astype(int)
df_encoded['race_White'] = (df['race'] == 'White').astype(int)
df_encoded['race_Black'] = (df['race'] == 'Black').astype(int)
df_encoded['race_Asian'] = (df['race'] == 'Asian').astype(int)

# Features for training
feature_cols = ['age', 'education_years', 'hours_per_week', 
                'gender_encoded', 'race_White', 'race_Black', 'race_Asian']
X = df_encoded[feature_cols]
y = df_encoded['high_income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

print(f"✅ Model trained!")
print(f"📊 Accuracy: {clf.score(X_test, y_test):.3f}")

✅ Model trained!
📊 Accuracy: 0.772


In [4]:
# Create DBDataset WITH protected attributes
dataset = DBDataset(
    data=df_encoded,
    target_column='high_income',
    model=clf,
    test_size=0.2,
    random_state=42,
    dataset_name='Adult Income Classification'
)

# Create Experiment with protected_attributes
exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    protected_attributes=['gender', 'race'],  # ← Specify protected attributes
    random_state=42
)

print("✅ Experiment created with protected attributes!")
print(f"🛡️  Protected attributes: {exp.protected_attributes}")

Feature names unseen at fit time:
- gender
- race

✅ Initial model evaluation complete: RandomForestClassifier
2025-11-04 22:47:21,688 - deepbridge.experiment - WARNING - Could not calculate ROC AUC for primary_model: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- gender
- race

✅ Experiment created with protected attributes!
🛡️  Protected attributes: ['gender', 'race']


/home/guhaase/projetos/DeepBridge/deepbridge/utils/feature_manager.py:25: UserWarning: Inferred 2 categorical features: gender, race
  warnings.warn(


## 4️⃣ Auto-detection of Sensitive Attributes

DeepBridge can automatically detect attributes that appear sensitive!

In [5]:
print("🔍 AUTO-DETECTION OF SENSITIVE ATTRIBUTES\n" + "="*60)

# If available
if hasattr(exp, 'detect_sensitive_attributes'):
    detected = exp.detect_sensitive_attributes()
    print(f"\n✅ Detected attributes: {detected}")
else:
    print("\n💡 DeepBridge automatically detects attributes such as:")
    print("   - Columns named: gender, sex, race, ethnicity, age")
    print("   - Categorical columns with few unique values")
    print("   - Columns with suspicious distribution")
    
print("\n📋 In our dataset:")
print(f"   ✅ 'gender' - Detected")
print(f"   ✅ 'race' - Detected")
print(f"   ✅ 'age' - Can be detected (age is sensitive in some contexts)")

🔍 AUTO-DETECTION OF SENSITIVE ATTRIBUTES


TypeError: Experiment.detect_sensitive_attributes() missing 1 required positional argument: 'dataset'

## 5️⃣ The 15 Fairness Metrics

DeepBridge calculates **15 different fairness metrics**!

### Main Categories:

#### 1. **Demographic Parity**
```
P(Ŷ=1 | A=0) = P(Ŷ=1 | A=1)
```
Positive prediction rate should be equal across groups

#### 2. **Equal Opportunity**
```
P(Ŷ=1 | Y=1, A=0) = P(Ŷ=1 | Y=1, A=1)
```
True Positive Rate equal across groups

#### 3. **Equalized Odds**
```
TPR and FPR equal across groups
```
Combines True Positive and False Positive

#### 4. **Disparate Impact**
```
Ratio = P(Ŷ=1 | A=unprivileged) / P(Ŷ=1 | A=privileged)
```
EEOC Rule: Ratio should be >= 0.8 (80%)

### Complete List:
1. Demographic Parity Difference
2. Demographic Parity Ratio
3. Equal Opportunity Difference
4. Equalized Odds Difference
5. Disparate Impact
6. Statistical Parity Difference
7. Average Odds Difference
8. Theil Index
9. False Positive Rate Difference
10. False Negative Rate Difference
11. Precision Difference
12. Recall Difference
13. F1 Score Difference
14. Accuracy Difference
15. Selection Rate

## 6️⃣ Execute First Fairness Test

In [ ]:
print("🧪 Running fairness test...\n")

fairness_result = exp.run_fairness_tests(config='quick')

print("\n✅ Fairness test completed!")

## 7️⃣ Verify EEOC Compliance (80% Rule)

In [ ]:
print("\n⚖️  EEOC 80% RULE (Four-Fifths Rule)\n" + "="*60)

print("\n📋 What is it?")
print("   The EEOC (USA) defines that the selection rate of the disadvantaged group")
print("   must be at least 80% of the privileged group's rate.")

print("\n📐 Formula:")
print("   Disparate Impact = P(Ŷ=1 | Unprivileged) / P(Ŷ=1 | Privileged)")
print("   ✅ PASS: >= 0.80")
print("   ❌ FAIL: < 0.80")

if hasattr(fairness_result, 'passes_eeoc_compliance'):
    passes = fairness_result.passes_eeoc_compliance()
    di_score = fairness_result.get('disparate_impact', {})
    
    print(f"\n📊 Result:")
    print(f"   Disparate Impact: {di_score}")
    print(f"   EEOC Compliance: {'✅ PASS' if passes else '❌ FAIL'}")
else:
    print("\n💡 Example analysis:")
    print("   If male rate = 50% and female rate = 35%")
    print("   Disparate Impact = 35% / 50% = 0.70")
    print("   Result: ❌ FAIL (< 0.80)")
    print("   Action: Model needs adjustment!")

## 8️⃣ First Metrics

In [ ]:
print("\n📊 FAIRNESS SUMMARY\n" + "="*60)

if hasattr(fairness_result, 'summary'):
    summary = fairness_result.summary()
    print(summary)
else:
    # Calculate manually for demonstration
    y_pred = clf.predict(X_test)
    
    # Reconstruct gender and race for test set
    test_indices = X_test.index
    test_gender = df_encoded.loc[test_indices, 'gender']
    test_race = df_encoded.loc[test_indices, 'race']
    
    # Rates by gender
    male_mask = test_gender == 'Male'
    female_mask = test_gender == 'Female'
    
    male_rate = y_pred[male_mask].mean()
    female_rate = y_pred[female_mask].mean()
    
    di_gender = female_rate / male_rate if male_rate > 0 else 0
    
    print("\n👥 GENDER:")
    print(f"   Positive prediction rate (Male): {male_rate:.1%}")
    print(f"   Positive prediction rate (Female): {female_rate:.1%}")
    print(f"   Disparate Impact: {di_gender:.3f}")
    print(f"   EEOC 80% Rule: {'✅ PASS' if di_gender >= 0.8 else '❌ FAIL'}")
    
    # Rates by race
    white_mask = test_race == 'White'
    black_mask = test_race == 'Black'
    
    white_rate = y_pred[white_mask].mean() if white_mask.sum() > 0 else 0
    black_rate = y_pred[black_mask].mean() if black_mask.sum() > 0 else 0
    
    di_race = black_rate / white_rate if white_rate > 0 else 0
    
    print("\n🌍 RACE:")
    print(f"   Positive prediction rate (White): {white_rate:.1%}")
    print(f"   Positive prediction rate (Black): {black_rate:.1%}")
    print(f"   Disparate Impact: {di_race:.3f}")
    print(f"   EEOC 80% Rule: {'✅ PASS' if di_race >= 0.8 else '❌ FAIL'}")

## 🎉 Conclusion

### What you learned:
- ✅ **Importance of Fairness** - Real cases of bias
- ✅ **Protected Attributes** - Gender, race, age, etc.
- ✅ **15 Metrics** - Different definitions of fairness
- ✅ **EEOC 80% Rule** - Regulatory compliance
- ✅ **Auto-detection** - Identify sensitive attributes
- ✅ **First Test** - Execute fairness analysis

### Why this matters:
- ⚖️ **Ethical** - Fair models for everyone
- 📜 **Legal** - Avoid regulatory violations
- 💼 **Business** - Avoid lawsuits and damage
- 🎯 **Technical** - Better models

### Main Regulations:
- **USA**: EEOC, Fair Lending Laws
- **Europe**: GDPR, AI Act
- **Brazil**: LGPD

### Next Steps:
- 📘 `02_complete_fairness_analysis.ipynb` ⭐⭐ - **CRITICAL**
  - Deep analysis of all 15 metrics
  - Complete EEOC compliance
  - Professional reports
- 📘 `03_bias_mitigation.ipynb` - How to correct bias

<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; border-left: 5px solid #ff9800;">
<b>⚠️  IMPORTANT:</b> For production models affecting people (credit, hiring, healthcare), fairness analysis is MANDATORY!
</div>

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4caf50;">
<b>💡 Remember:</b> "High accuracy doesn't mean fair model" - always validate fairness!
</div>